# Exercises    
    For each of the following questions, formulate a null and alternative hypothesis (be as specific as you can be), then give an example of what a true positive, true negative, type I and type II errors would look like. Note that some of the questions are intentionally phrased in a vague way. It is your job to reword these as more precise questions that could be tested.

1. Has the network latency gone up since we switched internet service providers?
- null : latency performance is not related to internet providers 
- althypo : latency performance is related to internet providers 
- truepos : avg latency from one company is significantly higher than the other
- trueneg : avg latency from one company is not significantly higher than the other
- type1 : latency is better with one company statistically, but in reality it isnt
- type2 : company doesn't seem to matter with our tests, but in reality actually does
2.  Is the website redesign any good? (just as a comment, what does good even mean? For this I am assuming amt of users visiting site)
- null : website views are not related to the website change
- althypo : website views are related to website change
- truepos : website views are higher with the redesigned group compared to original
- falsepos : website views on original are higher than the redesigned
- type1 : views are higher on the new design, but in actuality it's due to just more people visiting, unrelated to the design
- type2 : our redesign doesn't change our engagement, but in reality it increases user engagement and views. 
3. Is our television ad driving more sales?
- null: advertising campaign does not influence sales
- althypo : advertising campaign influenced sales
- truepos : advertising campaing did create a larger amount of sales
- falsepos : advertising campaign did not increase sales
- type1 : sales are up, but not due to the ad. maybe its the weather
- type2 : sales aren't up, but that's due to people not needing our product. The ad is getting new customers, they just dont increase our sales enough. 